In [2]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv("../community_policing_data.csv")

/var/folders/qg/k11w__bd5r3g9x3r6yr17bsw0000gn/T/ipykernel_45322/1476994813.py:1: DtypeWarning: Columns (5,10,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../community_policing_data.csv")


In [6]:
df.head()

,STOP_DATE,AGENCY NAME,LOCATION,JURISDICTION,REASON FOR STOP,PERSON TYPE,RACE,ETHNICITY,AGE,GENDER,...,ACTION TAKEN,SPECIFIC VIOLATION,VIRGINIA CRIME CODE,PERSON SEARCHED,VEHICLE SEARCHED,ADDITIONAL ARREST,FORCE USED BY OFFICER,FORCE USED BY SUBJECT,RESIDENCY,VIOLATION TYPE
0,7/1/2020,Broadway Police Department,S MAIN/FORTH ST,ROCKINGHAM CO,TRAFFIC VIOLATION,NaN,WHITE,NOT HISPANIC OR LATINO,18.0,MALE,...,WARNING ISSUED,46.2-833,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
1,7/1/2020,Isle Of Wight County Sheriff's Office,36.967676 -76.511763,ISLE OF WIGHT CO,TRAFFIC VIOLATION,NaN,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,26.0,MALE,...,WARNING ISSUED,46.2-804,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
2,7/1/2020,Isle Of Wight County Sheriff's Office,36.95375 -76.540314,ISLE OF WIGHT CO,EQUIPMENT VIOLATION,NaN,UNKNOWN,UNKNOWN,82.0,NaN,...,WARNING ISSUED,46.2-1157,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
3,7/1/2020,Isle Of Wight County Sheriff's Office,36.969732 -76.569873,ISLE OF WIGHT CO,EQUIPMENT VIOLATION,NaN,WHITE,UNKNOWN,61.0,MALE,...,WARNING ISSUED,46.2-1157,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
4,7/1/2020,Isle Of Wight County Sheriff's Office,36.949132 -76.596691,ISLE OF WIGHT CO,TRAFFIC VIOLATION,NaN,WHITE,NOT HISPANIC OR LATINO,37.0,MALE,...,CITATION/SUMMONS,46.2-1157,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
